In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import random
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import IPython.display as display
from PIL import Image
keras = tf.keras

BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000
IMG_SIZE = 512

train_dir = './data/pre_512/train_images/'
test_dir = './data/pre_512/test_images/'
#train_dir = './data/raw/train_images/'
#test_dir = './data/raw/test_images/'
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
lables = np.array([0,1,2,3,4])

In [ ]:
df = train_df.copy()
np.unique(df['diagnosis'].values,return_counts = True)

In [ ]:
def dumyfunc(dff, tr, val, classes):
    lx = dff['diagnosis'].isin(classes)
    df = dff[lx]
    tset = []
    vset = []
    for i in range(df.shape[0]):
        j = classes.index(df.iat[i,1])
        if val[j] != 0:
            val[j] -= 1
            vset.append(tuple((df.iat[i,0],j)))       
        elif tr[j] != 0:
            tr[j] -= 1
            tset.append(tuple((df.iat[i,0],j)))
    return tset, vset

def create_img_lab_pairs(pairs, path_dir, targetsize):
    imgs = []
    labels = []
    for pair in pairs:
        file_path = path_dir + pair[0] + ".png"
        img = tf.io.read_file(file_path)
        img = tf.image.decode_png(img, channels=3)
        img = tf.image.convert_image_dtype(img, tf.float32)
        img = (img/127.5) - 1
        #img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
        imgs.append(img)
        cat_val = np.zeros(targetsize,dtype = 'int')
        cat_val[pair[1]] = 1
        labels.append(cat_val)
    imgs = tf.data.Dataset.from_tensor_slices(imgs)
    labels = tf.data.Dataset.from_tensor_slices(np.array(labels))
    return tf.data.Dataset.zip((imgs,labels))

def prepare_input(df,valcnts,classes,dirpath):
    tset, vset = dumyfunc(df, valcnt, classes)
    random.shuffle(tset)
    tset = create_img_lab_pairs(tset, dirpath, len(classes))
    vset = create_img_lab_pairs(vset, dirpath, len(classes))
    return tset,vset

'''def InputGenerator(sset,sz):
    for i in range(0,len(sset),BATCH_SIZE):  
        imgs = []
        labels = []
        for j in range(i,i + BATCH_SIZE):
            pair = sset[j]
            file_path = train_dir + pair[0] + ".png"
            img = tf.io.read_file(file_path)
            img = tf.image.decode_png(img, channels=3)
            img = tf.image.convert_image_dtype(img, tf.float32)
            img = (img/127.5) - 1
            imgs.append(img)
            cat_val = np.zeros(targetsize,dtype = 'int')
            cat_val[pair[1]] = 1
            labels.append(cat_val)
        yield (imgs,labels)
        imgs = tf.data.Dataset.from_tensor_slices(imgs)
        labels = tf.data.Dataset.from_tensor_slices(np.array(labels))
        yield tf.data.Dataset.zip((imgs,labels))'''
vset = []
tset = []
targetsize = 2

def vgen():
    for pair in vset:  
        file_path = train_dir + pair[0] + ".png"
        img = tf.io.read_file(file_path)
        img = tf.image.decode_png(img, channels=3)
        img = tf.image.convert_image_dtype(img, tf.float32)
        img = (img/127.5) - 1
        #img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
        cat_val = np.zeros(targetsize,dtype = 'int')
        cat_val[pair[1]] = 1
        cat_val = 1
        if pair[1] == 0: 
            cat_val = 0
        yield tuple((img,cat_val))

def tgen():
    for pair in tset:  
        file_path = train_dir + pair[0] + ".png"
        img = tf.io.read_file(file_path)
        img = tf.image.decode_png(img, channels=3)
        img = tf.image.convert_image_dtype(img, tf.float32)
        img = (img/127.5) - 1
        #img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
        cat_val = np.zeros(targetsize,dtype = 'int')
        cat_val[pair[1]] = 1
        cat_val = 1
        if pair[1] == 0: 
            cat_val = 0
        yield tuple((img,cat_val))

In [ ]:
tset, vset = dumyfunc(train_df, [950,950], [49,49], [0,2])
for i in range(1000):
    random.shuffle(tset)
tbatches = tf.data.Dataset.from_generator(tgen,output_types = (tf.float32, tf.int32),output_shapes = ((512, 512, 3), ())).shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
vbatches = tf.data.Dataset.from_generator(vgen,output_types = (tf.float32, tf.int32),output_shapes = ((512, 512, 3), ())).batch(BATCH_SIZE)

In [ ]:
rr = np.array([x[1] for x in tset])
np.unique(rr,return_counts = True)

In [ ]:
'''for image_batch, label_batch in tbatches.take(1):
    pass

In [ ]:

input_batch = tf.keras.Input(shape = (IMG_SIZE,IMG_SIZE,3))

conv11_layer = tf.keras.layers.Conv2D(32,(3,3),activation = 'relu',input_shape=(IMG_SIZE,IMG_SIZE,3),padding = "same")
conv11_batch = conv11_layer(input_batch)

conv12_layer = tf.keras.layers.Conv2D(32,(5,5),activation = 'relu',input_shape=(IMG_SIZE,IMG_SIZE,3),padding = "same")
conv12_batch = conv12_layer(input_batch)

add1_layer = tf.keras.layers.Concatenate()
add1_batch = add1_layer([conv11_batch, conv12_batch])

maxPool1_layer = tf.keras.layers.MaxPooling2D((2,2))
maxPool1_batch = maxPool1_layer(add1_batch)

conv21_layer = tf.keras.layers.Conv2D(32,(3,3),activation = 'relu',padding = "same")
conv21_batch = conv21_layer(maxPool1_batch)

conv22_layer = tf.keras.layers.Conv2D(32,(5,5),activation = 'relu',padding = "same")
conv22_batch = conv22_layer(maxPool1_batch)

add2_layer = tf.keras.layers.Concatenate()
add2_batch = add1_layer([conv21_batch, conv22_batch])

maxPool2_layer = tf.keras.layers.MaxPooling2D((2,2))
maxPool2_batch = maxPool2_layer(add2_batch)

conv3_layer = tf.keras.layers.Conv2D(128,(3,3),activation = 'relu')
conv3_batch = conv3_layer(maxPool2_batch)

maxPool3_layer = tf.keras.layers.MaxPooling2D((2,2))
maxPool3_batch = maxPool3_layer(conv3_batch)

conv4_layer = tf.keras.layers.Conv2D(128,(3,3),activation = 'relu')
conv4_batch = conv4_layer(maxPool3_batch)

maxPool4_layer = tf.keras.layers.MaxPooling2D((2,2))
maxPool4_batch = maxPool4_layer(conv4_batch)

flatten_layer = tf.keras.layers.Flatten()
flatten_batch = flatten_layer(maxPool3_batch)

dense1_layer = tf.keras.layers.Dense(64,activation = 'relu')
dense1_batch = dense1_layer(flatten_batch)

#dropout_layer2 = keras.layers.Dropout(rate = drop_out_rate)
#dropout_batch2 = dropout_layer2(dense1_batch)

prediction_layer = tf.keras.layers.Dense(1,activation='sigmoid')
prediction_batch = prediction_layer(dense1_batch)
model = tf.keras.Model(inputs = input_batch, outputs = prediction_batch)

base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

model.summary()

In [ ]:
initial_epochs = 5
validation_steps=10
history = model.fit(tbatches,
                    epochs=initial_epochs,
                    validation_data=vbatches)

In [ ]:
drop_out_rate = 0.4

conv1_layer = tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',input_shape=(IMG_SIZE,IMG_SIZE,3))
conv1_batch = conv1_layer(image_batch)

maxPool1_layer = tf.keras.layers.MaxPooling2D((2,2))
maxPool1_batch = maxPool1_layer(conv1_batch)

conv2_layer = tf.keras.layers.Conv2D(128,(3,3),activation = 'relu')
conv2_batch = conv2_layer(maxPool1_batch)

maxPool2_layer = tf.keras.layers.MaxPooling2D((2,2))
maxPool2_batch = maxPool2_layer(conv2_batch)

conv3_layer = tf.keras.layers.Conv2D(256,(3,3),activation = 'relu')
conv3_batch = conv3_layer(maxPool2_batch)

maxPool3_layer = tf.keras.layers.MaxPooling2D((2,2))
maxPool3_batch = maxPool3_layer(conv3_batch)

'''conv4_layer = tf.keras.layers.Conv2D(256,(3,3),activation = 'relu')
conv4_batch = conv4_layer(maxPool3_batch)

maxPool4_layer = tf.keras.layers.MaxPooling2D((2,2))
maxPool4_batch = maxPool2_layer(conv4_batch)

conv5_layer = tf.keras.layers.Conv2D(512,(3,3),activation = 'relu')
conv5_batch = conv5_layer(maxPool4_batch)

maxPool5_layer = tf.keras.layers.MaxPooling2D((2,2))
maxPool5_batch = maxPool5_layer(conv5_batch)'''

average_layer = tf.keras.layers.GlobalAveragePooling2D()
average_batch = average_layer(maxPool3_batch)

#flatten_layer = tf.keras.layers.Flatten()
#flatten_batch = flatten_layer(maxPool3_batch)



dense1_layer = tf.keras.layers.Dense(512,activation = 'relu')
dense1_batch = dense1_layer(average_batch)

dropout_layer2 = keras.layers.Dropout(rate = drop_out_rate)
dropout_batch2 = dropout_layer2(dense1_batch)

prediction_layer = tf.keras.layers.Dense(2)
prediction_batch = prediction_layer(dropout_batch2)
'''conv4_layer, maxPool4_layer, conv5_layer, maxPool5_layer, flatten_layer,'''
model = tf.keras.Sequential([conv1_layer, maxPool1_layer, conv2_layer, maxPool2_layer, conv3_layer, maxPool3_layer,
                             average_layer,
                             dense1_layer, dropout_layer2, prediction_layer])

base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses. CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
'''model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])'''
model.summary()

In [ ]:
initial_epochs = 5
validation_steps=20
history = model.fit(tbatches,
                    epochs=initial_epochs,
                    validation_data=vbatches)